In [1]:
# Suzan Iloglu, May 21,2020
# Import packages
import csv
import gurobipy as gp
from itertools import product
import geopandas as gpd
import pandas as pd
import numpy as np
import math
import time
import requests
import io
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
pd.options.display.max_columns =200
from IPython.display import Image


# MAPPING THE NEW POLITICS OF CARE: COMMUNITY HEALTH WORKERS
The project presents multiple options for how individual workers in such a Community Health Corps might be distributed within each state. It shows that what you choose to prioritize greatly impacts where care would be sent. We can define communities in greatest need in many ways: we can think about our current crisis and send people to where the COVID19 pandemic rages most fiercely; we can think of long term measures of social and economic inequality embedded in metrics like the Centers for Disease Control and Prevention’s Social Vulnerability Index; we can focus on the places with too many people dying too young and use the County Health Rankings Years-of-Potential-Life-Lost measure; we can think of joblessness and how the pandemic has thrown many into unemployment and target our resources in this way. 

The followings are our options to choose to define vulnerability:


- SOCIAL VULNERABILITY INDEX
- MEDICAID 
- UNEMPLOYMENT
- YEARS OF POTENTIAL LIFE LOST
- TOTAL COVID CASES
- COVID CASES BY POPULATION
- COVID DEATHS BY POPULATION

We will start with Social Vulnerability Index (SVI) from CDC website.

### I. Importing SVI data which includes the variables for calculating county SVI for each state
The CDC uses both a USA-wide and a state by state SVI scores. For our project given that funding is likely going to be managed at a state level, using a state by state SVI scores makes the most sense and will be most sensitive to regional socioeconomic differences. Even though the CDC SVI scores are calculated using percentile rankings, the data sets include raw data estimates for each variables. The following table shows the variablaes used in the method of calculating SVI scores. 




      American Community Survey (ACS), 2014-2018 (5-year) data for the following estimates:
<img src="Data/img/SVI_comp.png" width="500">


Note: Full documentation for 2018 data is available <a href="https://svi.cdc.gov/data-and-tools-download.html">here</a> 
This part of the code shows preliminary mapping of <a href = "https://svi.cdc.gov/">the CDC's Social Vulnerability Index</a>.

Later in the notebook, we will provide the formula to create the SVI value we use in our project. First, we import the data for the US mainland and Puerto Rico.

In [2]:
## import svi data downloaded from CDC website as cited above

## 48 state SVI scores by county
svi_counties_mainland = gpd.read_file("Data/SVI2018_US_COUNTY/SVI2018_US_county.shp")

## Puerto Rico SVI scores by county
svi_counties_puerto_rico = gpd.read_file("Data/PuertoRico_COUNTY/SVI2018_PuertoRico_county.shp")

## Merge 48 states and Puerto Rico SVI 
svi_counties = pd.concat([svi_counties_mainland,svi_counties_puerto_rico ], sort = False)


In [3]:
## Replacing -999 values with 0 for calculations
svi_county = svi_counties.fillna(0)
svi_county  = svi_county.replace(-999, 0)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)

In [4]:
## Create the list for State
State = svi_county.STATE.unique().tolist()

In [5]:
# Create a seperate dictionary for the variables to calculate SVI

# Persons below poverty estimate, 2014-2018 ACS
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Civilian (age 16+) unemployed estimate, 2014-2018 ACS
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.E_UNEMP))

# Per capita income estimate, 2014-2018 ACS
E_PCI = dict(zip(svi_county.FIPS, svi_county.E_PCI))

# Persons (age 25+) with no high school diploma estimate, 2014-2018 ACS
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.E_NOHSDP))

# Persons aged 65 and older estimate
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.E_AGE65))

# Persons aged 17 and younger estimate
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.E_AGE17))

# Population with a disability estimate
E_DISABL = dict(zip(svi_county.FIPS, svi_county.E_DISABL))

# Single parent households with children under 18 estimate
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.E_SNGPNT))

# Minority (all persons except white, nonHispanic) estimate, 2014-2018 ACS
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.E_MINRTY))

# Persons (age 5+) who speak English "less than well" estimate, 2014-2018 ACS
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.E_LIMENG))

# Housing in structures with 10 or more units estimate, 2014-2018 ACS
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.E_MUNIT))

# Mobile homes estimate MOE, 2014-2018 ACS
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.E_MOBILE))

# At household level (occupied housing units), more people than rooms estimate, 2014-2018 ACS
E_CROWD = dict(zip(svi_county.FIPS, svi_county.E_CROWD))

# Households with no vehicle available estimate, 2014-2018 ACS
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.E_NOVEH))

# Persons in institutionalized group quarters estimate, 2014-2018 ACS
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.E_GROUPQ))

# Percentage of persons below poverty estimate
E_POV = dict(zip(svi_county.FIPS, svi_county.E_POV))

# Medicaid 
Medicaid is a means-tested health insurance program for low-income children, pregnant women, adults, seniors, and people with disabilities. Medicaid is jointly funded by federal and state governments and managed by states within federal standards and a wide range of state options. <a href="https://data.medicaid.gov/Enrollment/State-Medicaid-and-CHIP-Applications-Eligibility-D/n5ce-jxme"> Data Source for Medicaid Enrollment </a> 

In [6]:

import sodapy
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.medicaid.gov", None)


# Returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("83yt-67it", limit=4000)


# Read the medicaid demand data
#df_mm = pd.read_csv("Data/2020_06_Preliminary_applications__eligibility_determinations__and_enrollment_data.csv")

# Convert to pandas DataFrame
df_mm = pd.DataFrame.from_records(results)
df_mm.head(5)
df_mm.columns


Index(['applications_for_financial_assistance_submitted_to_the_state_based_marketplace',
       'applications_for_financial_assistance_submitted_to_the_state_based_marketplace_footnotes',
       'final_report', 'geocoded_column',
       'individuals_determined_eligible_for_chip_at_application',
       'individuals_determined_eligible_for_chip_at_application_footnotes',
       'individuals_determined_eligible_for_medicaid_at_application',
       'individuals_determined_eligible_for_medicaid_at_application_footnotes',
       'latitude', 'longitude', 'medicaid_and_chip_child_enrollment',
       'medicaid_and_chip_child_enrollment_footnotes',
       'new_applications_submitted_to_medicaid_and_chip_agencies',
       'new_applications_submitted_to_medicaid_and_chip_agencies_footnotes',
       'preliminary_updated', 'report_date', 'state_abbreviation',
       'state_expanded_medicaid', 'state_name',
       'total_applications_for_financial_assistance_submitted_at_state_level',
       'total_a

In [7]:
df_mm['State Name'] = df_mm['state_name'].str.upper() 

In [8]:
Medicaid_state = dict(zip(df_mm['State Name'], df_mm['total_medicaid_and_chip_enrollment']))
Medicaid_state['PUERTO RICO'] = 1622194
print (Medicaid_state)

{'ALABAMA': '954438', 'ALASKA': '231145', 'ARIZONA': '1839932', 'ARKANSAS': '830467', 'CALIFORNIA': '11847711', 'COLORADO': '1337805', 'CONNECTICUT': '874974', 'DELAWARE': '239009', 'DISTRICT OF COLUMBIA': '250860', 'FLORIDA': '3892552', 'GEORGIA': '1928703', 'HAWAII': '351337', 'IDAHO': '340742', 'ILLINOIS': '2987496', 'INDIANA': '1602976', 'IOWA': '699741', 'KANSAS': '401103', 'KENTUCKY': '1416013', 'LOUISIANA': '1585024', 'MAINE': '232455', 'MARYLAND': '1372695', 'MASSACHUSETTS': '1616404', 'MICHIGAN': '2439425', 'MINNESOTA': '1085778', 'MISSISSIPPI': '632427', 'MISSOURI': '923641', 'MONTANA': '257006', 'NEBRASKA': '254159', 'NEVADA': '685073', 'NEW HAMPSHIRE': '193436', 'NEW JERSEY': '1759653', 'NEW MEXICO': '772102', 'NEW YORK': '6263164', 'NORTH CAROLINA': '1851558', 'NORTH DAKOTA': '96757', 'OHIO': '2788134', 'OKLAHOMA': '797220', 'OREGON': '1053931', 'PENNSYLVANIA': '3069309', 'RHODE ISLAND': '305208', 'SOUTH CAROLINA': '1048276', 'SOUTH DAKOTA': '114059', 'TENNESSEE': '1489536

In [9]:
df_mmm = pd.read_csv("Data/ACSST5Y2019.S2704_data_with_overlays_2020-12-11T225641.csv", header=[1])
df_mmm.head(1)
#df_mmm.dtypes

,id,Geographic Area Name,Estimate!!Total!!Civilian noninstitutionalized population,Margin of Error!!Total!!Civilian noninstitutionalized population,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicare coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination!!65 years and over,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!Under 19,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!19 to 64 years,Estimate!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Margin of Error!!Total!!COVERAGE ALONE OR IN COMBINATION!!VA health care coverage alone or in combination!!65 years and over,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Below 138 percent of the poverty threshold,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!At or above 138 percent of the poverty threshold,"Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)","Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Worked full-time, year-round (19-64 years)",Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!Under 6,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Margin of Error!!Total!!PUBLIC HEALTH INSURANCE ALONE OR IN COMBINATION!!6 to 18 years,Estimate!!Total!!PUBLIC HEALTH INSURANCE ALON

In [10]:
df_mmm['FIPS'] = df_mmm.id.astype(str).str[9:]
df_mmm.head(5)
df_mmm.FIPS.astype(int)
df_mmm['FIPS'] = pd.to_numeric(df_mmm['FIPS'])

In [11]:
# 2018 ACI data
# ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!Medicaid/means-tested public coverage alone or in combination"]))

#2019 ACI data
ACI_total  = dict(zip(df_mmm['FIPS'], df_mmm["Estimate!!Public Coverage!!COVERAGE ALONE OR IN COMBINATION!!Medicaid/means-tested public coverage alone or in combination"]))




In [12]:

#df_m = pd.read_csv("Data/Medicaid_Demand.csv")

# Unemployment 
The unemployment rate is calculated by the U.S. Bureau of Labor Statistics as the percentage of the civilian labor force who are without jobs and have actively sought work within the past four weeks. <a href="https://www.bls.gov/lau/laufaq.htm#Q01"> Data Source for Unemployment  </a> 

In [13]:
from io import StringIO
import datetime 
from datetime import date
from dateutil.relativedelta import relativedelta

back = date.today() + relativedelta(months= -3)
three_months_ago = back.strftime('%b-%y') 


url = 'https://www.bls.gov/web/metro/laucntycur14.txt'
s = requests.get(url).text



df_unemp = pd.read_csv(StringIO(s), sep='|',  skiprows=7, skipfooter=6, engine='python', names = ['LAUS Area Code', 'FIPS State', 'FIPS County', 'Area Title', 'Period', 'Civilian Labor Force','Employed','Unemployed_Level','Unemployed_Rate'])

df_unemp['Period'] = df_unemp['Period'].astype(str)

df_unemp['FIPS'] = df_unemp['LAUS Area Code'].str[3:8]

df_unemp = df_unemp[df_unemp['Period'].str.contains(str(three_months_ago))]

df_unemp.head(5)
#df_unemp.dtypes

,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
38627,CN0100100000000,1,1,"Autauga County, AL",Feb-21,"25,563","24,783",780,3.1,01001
38628,CN0100300000000,1,3,"Baldwin County, AL",Feb-21,"95,716","92,589","3,127",3.3,01003
38629,CN0100500000000,1,5,"Barbour County, AL",Feb-21,"8,495","7,972",523,6.2,01005
38630,CN0100700000000,1,7,"Bibb County, AL",Feb-21,"8,590","8,280",310,3.6,01007
38631,CN0100900000000,1,9,"Blount County, AL",Feb-21,"24,750","24,208",542,2.2,01009


In [14]:
#df_unemp[[ 'FIPS', 'Area Title', 'Period','Unemployed_Level','Unemployed_Rate']].to_csv('Data/County_employment.csv', index=False)


In [15]:


df_unemp.replace({'-', 0})

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.replace(',', '')

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].astype(str)

df_unemp['Unemployed_Level'] = df_unemp['Unemployed_Level'].str.strip()

df_unemp['Unemployed_Rate'] = df_unemp['Unemployed_Rate'].str.strip()

df_unemp['FIPS'] = pd.to_numeric(df_unemp['FIPS'])

df_unemp['Unemployed_Level'] = pd.to_numeric(df_unemp['Unemployed_Level'])

df_unemp['Unemployed_Rate'] = pd.to_numeric(df_unemp['Unemployed_Rate'])




# Fill NA with 0
#df_unemp = df_unemp.fillna(0)
df_unemp.tail(5)
#df_unemp.dtypes


,LAUS Area Code,FIPS State,FIPS County,Area Title,Period,Civilian Labor Force,Employed,Unemployed_Level,Unemployed_Rate,FIPS
41841,CN7214500000000,72,145,"Vega Baja Municipio, PR",Feb-21,"12,686","11,429",1257,9.9,72145
41842,CN7214700000000,72,147,"Vieques Municipio, PR",Feb-21,"2,475","2,183",292,11.8,72147
41843,CN7214900000000,72,149,"Villalba Municipio, PR",Feb-21,"6,835","6,021",814,11.9,72149
41844,CN7215100000000,72,151,"Yabucoa Municipio, PR",Feb-21,"8,240","7,352",888,10.8,72151
41845,CN7215300000000,72,153,"Yauco Municipio, PR",Feb-21,"8,956","7,900",1056,11.8,72153


In [16]:
 
## Read the Unemployment data using cvs data 
#df_unemp = pd.read_csv("Data/Unemployment.csv")

# Fill NA with 0
#df_unemp = df_unemp.fillna(0)

#df_unemp.head(5)

# Years of Potential Life Lost (YPLL)

Years of Potential Life Lost (YPLL) measures the rate of premature deaths by region. YPLL is calculated as the sum of the estimated number of years that individuals would have lived if they had not died before the age of 75 per 100,000 people. <a href="https://www.countyhealthrankings.org/sites/default/files/media/document/2020%20County%20Health%20Rankings%20Data%20-%20v2.xlsx"> Data Source for YPLL.  </a> More information about YPLL can be dounf in this <a href="https://www.countyhealthrankings.org/explore-health-rankings/measures-data-sources/county-health-rankings-model/health-outcomes/length-of-life/premature-death-ypll"> link. </a> 


In [17]:
# Read the YPLL data
df_y = pd.read_csv("Data/YPLL.csv")

# Fill NA with the mean of the data
df_y = df_y.fillna(df_y.mean())


# Population

In [18]:
# Read the Population data
df_pop = pd.read_csv("Data/County_pop_2019.csv")

# Fill NA with 0
df_pop = df_pop.fillna(0)


In [19]:
# Create a dictionary for the county and population
population_county = df_pop.set_index('FIPS')['pop'].to_dict()

# Create a dictionary for the county and YPLL
YPLL = dict(zip(df_y.FIPS, df_y.YPLL))

# Create a dictionary for the county and Unemployment
Unemployment = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Level))


# Create a dictionary for the county and Community Health Workers (CHW) demand
# Note that we assume a CHW can serve 55 Medicaid patient so the demand for CHW will be

#Medicaid_demand = dict(zip(df_m.FIPS, df_m.Med_Demand))


In [20]:
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

# COVID-19 Cases & COVID-19 Cases per Capita

What are COVID-19 Cases and COVID-19 Cases per Capita?

COVID-19 cases is an absolute metric of the total number of COVID-19 cases in a county over the last fourteen days.  COVID-19 cases per 100,000 is a relative metric calculated by dividing the number of COVID-19 cases by the estimated county population and multiplying by 100,000.  Cases include both confirmed cases, based on viral testing, and probable cases, based on specific criteria for symptoms and epidemiological exposure. We use NY Times Covid data. 


In [21]:
#### Data with the most recent date in NY Times dataset:

today = time.strftime('%Y-%m-%d')
covid_data_update_date = today#'2020-07-21'#today #or enter a specific date such as '2020-07-06'


In [22]:
## 14 day period defined
data_date_dt = pd.to_datetime(covid_data_update_date,infer_datetime_format = True)

N = 14

date_N_days_ago = data_date_dt - timedelta(days = N)

date_N1_days_ago = data_date_dt - timedelta(days = N+1)

In [23]:

# URL for mainland US data
url = "http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
s = requests.get(url).content
covid = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [24]:
covid.tail(100)

,date,county,state,fips,cases,deaths
1355364,2021-05-25,Webster,West Virginia,54101.0,510,8.0
1355365,2021-05-25,Wetzel,West Virginia,54103.0,1372,31.0
1355366,2021-05-25,Wirt,West Virginia,54105.0,444,3.0
1355367,2021-05-25,Wood,West Virginia,54107.0,7868,172.0
1355368,2021-05-25,Wyoming,West Virginia,54109.0,2020,39.0
1355369,2021-05-25,Adams,Wisconsin,55001.0,1950,18.0
1355370,2021-05-25,Ashland,Wisconsin,55003.0,1385,15.0
1355371,2021-05-25,Barron,Wisconsin,55005.0,6022,83.0
1355372,2021-05-25,Bayfield,Wisconsin,55007.0,1284,19.0
1355373,2021-05-25,Brown,Wisconsin,55009.0,35417,290.0


Note: Since NY data is seperately available, we first read the NY data for all 5 different borough then combine with the rest of US data.

In [25]:
# URL for NY
url = "https://raw.githubusercontent.com/nychealth/coronavirus-data/master/trends/data-by-day.csv"
#"https://raw.githubusercontent.com/nychealth/coronavirus-data/master/data-by-day.csv"

ny = requests.get(url).content
covid_ny = pd.read_csv(io.StringIO(ny.decode('utf-8')))


covid_ny.tail(5)

,date_of_interest,CASE_COUNT,PROBABLE_CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,PROBABLE_DEATH_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,HOSP_COUNT_7DAY_AVG,DEATH_COUNT_7DAY_AVG,ALL_DEATH_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_HOSPITALIZED_COUNT,BX_DEATH_COUNT,BX_PROBABLE_DEATH_COUNT,BX_CASE_COUNT_7DAY_AVG,BX_ALL_CASE_COUNT_7DAY_AVG,BX_HOSPITALIZED_COUNT_7DAY_AVG,BX_DEATH_COUNT_7DAY_AVG,BX_ALL_DEATH_COUNT_7DAY_AVG,BK_CASE_COUNT,BK_PROBABLE_CASE_COUNT,BK_HOSPITALIZED_COUNT,BK_DEATH_COUNT,BK_PROBABLE_DEATH_COUNT,BK_CASE_COUNT_7DAY_AVG,BK_ALL_CASE_COUNT_7DAY_AVG,BK_HOSPITALIZED_COUNT_7DAY_AVG,BK_DEATH_COUNT_7DAY_AVG,BK_ALL_DEATH_COUNT_7DAY_AVG,MN_CASE_COUNT,MN_PROBABLE_CASE_COUNT,MN_HOSPITALIZED_COUNT,MN_DEATH_COUNT,MN_PROBABLE_DEATH_COUNT,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,MN_HOSPITALIZED_COUNT_7DAY_AVG,MN_DEATH_COUNT_7DAY_AVG,MN_ALL_DEATH_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_HOSPITALIZED_COUNT,QN_DEATH_COUNT,QN_PROBABLE_DEATH_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,QN_HOSPITALIZED_COUNT_7DAY_AVG,QN_DEATH_COUNT_7DAY_AVG,QN_ALL_DEATH_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_HOSPITALIZED_COUNT,SI_DEATH_COUNT,SI_PROBABLE_DEATH_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,SI_HOSPITALIZED_COUNT_7DAY_AVG,SI_DEATH_COUNT_7DAY_AVG,SI_ALL_DEATH_COUNT_7DAY_AVG,INCOMPLETE
444,05/18/2021,392,138,37,11,1,416,563,44,16,17,65,23,8,1,0,72,101,9,3,3,134,48,10,5,0,134,176,14,6,6,56,14,6,3,0,53,73,6,2,2,96,40,12,2,0,118,160,12,5,5,41,12,1,0,0,39,53,3,1,1,11000
445,05/19/2021,349,155,47,7,5,384,528,43,15,17,69,26,15,2,0,66,95,10,3,3,121,66,16,4,0,125,169,13,6,6,35,22,6,1,0,49,67,6,2,2,95,26,9,0,0,110,149,12,4,4,29,10,1,0,0,33,47,2,0,0,11000
446,05/20/2021,352,90,43,12,1,364,501,43,13,15,58,16,8,2,0,62,89,10,2,2,121,33,21,5,0,122,164,14,5,5,51,15,6,3,0,46,64,6,2,2,93,19,7,0,0,103,139,11,3,3,28,6,1,2,0,31,44,2,1,1,11000
447,05/21/2021,316,76,27,8,5,343,463,39,11,13,45,15,4,2,0,58,80,8,2,2,105,26,8,2,0,117,156,14,4,4,45,6,6,0,0,45,60,5,1,1,94,17,6,2,0,95,125,10,2,2,27,7,3,2,0,28,40,2,1,1,11000
448,05/22/2021,196,75,1,5,4,331,447,34,10,13,32,22,0,0,0,56,79,7,2,2,65,21,0,1,0,113,151,12,4,4,22,9,0,1,0,44,58,5,1,1,58,16,1,3,0,90,116,9,2,2,19,3,0,0,0,28,39,1,1,1,11000


In [26]:
Kings = covid_ny[['date_of_interest', 'BK_CASE_COUNT', 'BK_DEATH_COUNT']]
Kings.rename(columns = {'BK_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BK_DEATH_COUNT': 'deaths'} , inplace=True)
Kings['county'] = 'Kings'
Kings['state'] = 'New York'
Kings['fips'] = 36047.0
#Kings.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [27]:
Bronx = covid_ny[['date_of_interest', 'BX_CASE_COUNT', 'BX_DEATH_COUNT']]
Bronx.rename(columns = {'BX_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'BX_DEATH_COUNT': 'deaths'} , inplace=True)
Bronx['state'] = 'New York'
Bronx['county'] = 'Bronx'
Bronx['fips'] = 36005.0
#Bronx.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [28]:
Manhattan = covid_ny[['date_of_interest', 'MN_CASE_COUNT', 'MN_DEATH_COUNT']]
Manhattan.rename(columns = {'MN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'MN_DEATH_COUNT': 'deaths'} , inplace=True)
Manhattan['state'] = 'New York'
Manhattan['county'] = 'Manhattan'
Manhattan['fips'] = 36061.0
#Manhattan.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [29]:
Queens = covid_ny[['date_of_interest', 'QN_CASE_COUNT', 'QN_DEATH_COUNT']]
Queens.rename(columns = {'QN_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'QN_DEATH_COUNT': 'deaths'} , inplace=True)
Queens['state'] = 'New York'
Queens['county'] = 'Queens'
Queens['fips'] = 36081.0
#Queens.head(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [30]:
Richmond = covid_ny[['date_of_interest', 'SI_CASE_COUNT', 'SI_DEATH_COUNT']]
Richmond.rename(columns = {'SI_CASE_COUNT': 'cases', 'date_of_interest': 'date', 'SI_DEATH_COUNT': 'deaths'} , inplace=True)
Richmond['state'] = 'New York'
Richmond['county'] = 'Richmond'
Richmond['fips'] = 36085.0

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A 

In [31]:
covid.head(5)


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [32]:
Bronx.tail(5)

,date,cases,deaths,state,county,fips
444,05/18/2021,65,1,New York,Bronx,36005.0
445,05/19/2021,69,2,New York,Bronx,36005.0
446,05/20/2021,58,2,New York,Bronx,36005.0
447,05/21/2021,45,2,New York,Bronx,36005.0
448,05/22/2021,32,0,New York,Bronx,36005.0


In [33]:
#Kings['deaths'] = Kings.groupby(by=['fips'])['deaths_d'].sum()
#Bronx['deaths'] = Bronx.groupby(by=['fips'])['deaths_d'].sum()
#Manhattan['deaths'] = Manhattan.groupby(by=['fips'])['deaths_d'].sum()
#Queens['deaths'] = Queens.groupby(by=['fips'])['deaths_d'].sum()
#Richmond['deaths'] = Richmond.groupby(by=['fips'])['deaths_d'].sum()

In [34]:
Kings.head(5)

,date,cases,deaths,county,state,fips
0,02/29/2020,0,0,Kings,New York,36047.0
1,03/01/2020,0,0,Kings,New York,36047.0
2,03/02/2020,0,0,Kings,New York,36047.0
3,03/03/2020,0,0,Kings,New York,36047.0
4,03/04/2020,1,0,Kings,New York,36047.0


In [35]:
#Kings = Kings.drop(['deaths_d'], axis=1)
#Bronx = Bronx.drop(['deaths_d'], axis=1)
#Manhattan = Manhattan.drop(['deaths_d'], axis=1)
#Queens = Queens.drop(['deaths_d'], axis=1)
#Richmond = Richmond.drop(['deaths_d'], axis=1)

In [36]:
Kings.tail(5)

,date,cases,deaths,county,state,fips
444,05/18/2021,134,5,Kings,New York,36047.0
445,05/19/2021,121,4,Kings,New York,36047.0
446,05/20/2021,121,5,Kings,New York,36047.0
447,05/21/2021,105,2,Kings,New York,36047.0
448,05/22/2021,65,1,Kings,New York,36047.0


In [37]:
covid['dt'] = pd.to_datetime(covid['date'], infer_datetime_format=True)
Kings['dt'] = pd.to_datetime(Kings['date'], infer_datetime_format=True)
Bronx['dt'] = pd.to_datetime(Bronx['date'], infer_datetime_format=True)
Manhattan['dt'] = pd.to_datetime(Manhattan['date'], infer_datetime_format=True)
Queens['dt'] = pd.to_datetime(Queens['date'], infer_datetime_format=True)
Richmond['dt'] = pd.to_datetime(Richmond['date'], infer_datetime_format=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [38]:

Kings_c = Kings[(Kings['dt']>date_N1_days_ago) & (Kings['dt']<= data_date_dt)].copy()
Queens_c = Queens[(Queens['dt']>date_N1_days_ago) & (Queens['dt']<= data_date_dt)].copy()
Bronx_c = Bronx[(Bronx['dt']>date_N1_days_ago) & (Bronx['dt']<= data_date_dt)].copy()
Manhattan_c = Manhattan[(Manhattan['dt']>date_N1_days_ago) & (Manhattan['dt']<= data_date_dt)].copy()
Richmond_c = Richmond[(Richmond['dt']>date_N1_days_ago) & (Richmond['dt']<= data_date_dt)].copy()

In [39]:
#Kings['cases'] = Kings.loc[(Kings['dt']<= data_date_dt)].groupby(['dt'])['d_cases'].sum()
#Kings['cases'] = Kings.apply(lambda x: x[(Kings['dt']<= data_date_dt)]['d_cases'].sum())

In [40]:
Kings.head(50)

,date,cases,deaths,county,state,fips,dt
0,02/29/2020,0,0,Kings,New York,36047.0,2020-02-29
1,03/01/2020,0,0,Kings,New York,36047.0,2020-03-01
2,03/02/2020,0,0,Kings,New York,36047.0,2020-03-02
3,03/03/2020,0,0,Kings,New York,36047.0,2020-03-03
4,03/04/2020,1,0,Kings,New York,36047.0,2020-03-04
5,03/05/2020,3,0,Kings,New York,36047.0,2020-03-05
6,03/06/2020,1,0,Kings,New York,36047.0,2020-03-06
7,03/07/2020,2,0,Kings,New York,36047.0,2020-03-07
8,03/08/2020,5,0,Kings,New York,36047.0,2020-03-08
9,03/09/2020,16,0,Kings,New York,36047.0,2020-03-09


In [41]:
K_c = Kings_c.groupby(['fips']).sum().reset_index()
Q_c = Queens_c.groupby(['fips']).sum().reset_index()
B_c = Bronx_c.groupby(['fips']).sum().reset_index()
M_c = Manhattan_c.groupby(['fips']).sum().reset_index()
R_c = Richmond_c.groupby(['fips']).sum().reset_index()
K_c.head(19)

,fips,cases,deaths
0,36047.0,1351,54


In [42]:
covid_death = covid[(covid['dt'] >= data_date_dt- timedelta(days = 1))]
Kings_death = Kings.groupby(by=['fips'])['deaths'].sum().reset_index()
Bronx_death = Bronx.groupby(by=['fips'])['deaths'].sum().reset_index()
Manhattan_death = Manhattan.groupby(by=['fips'])['deaths'].sum().reset_index()
Queens_death = Queens.groupby(by=['fips'])['deaths'].sum().reset_index()
Richmond_death = Richmond.groupby(by=['fips'])['deaths'].sum().reset_index()

In [43]:
# We merge the NY data with the rest of the US data
covid = pd.concat([covid, Kings, Bronx, Manhattan, Queens, Richmond], sort = False)


In [44]:
Queens.tail(14)

,date,cases,deaths,state,county,fips,dt
435,05/09/2021,107,8,New York,Queens,36081.0,2021-05-09
436,05/10/2021,173,9,New York,Queens,36081.0,2021-05-10
437,05/11/2021,184,3,New York,Queens,36081.0,2021-05-11
438,05/12/2021,151,3,New York,Queens,36081.0,2021-05-12
439,05/13/2021,142,8,New York,Queens,36081.0,2021-05-13
440,05/14/2021,152,8,New York,Queens,36081.0,2021-05-14
441,05/15/2021,92,5,New York,Queens,36081.0,2021-05-15
442,05/16/2021,81,5,New York,Queens,36081.0,2021-05-16
443,05/17/2021,111,1,New York,Queens,36081.0,2021-05-17
444,05/18/2021,96,2,New York,Queens,36081.0,2021-05-18


In [45]:

covid.tail(50)

,date,county,state,fips,cases,deaths,dt
399,04/03/2021,Richmond,New York,36085.0,201,0.0,2021-04-03
400,04/04/2021,Richmond,New York,36085.0,142,3.0,2021-04-04
401,04/05/2021,Richmond,New York,36085.0,286,5.0,2021-04-05
402,04/06/2021,Richmond,New York,36085.0,248,4.0,2021-04-06
403,04/07/2021,Richmond,New York,36085.0,227,8.0,2021-04-07
404,04/08/2021,Richmond,New York,36085.0,234,3.0,2021-04-08
405,04/09/2021,Richmond,New York,36085.0,254,1.0,2021-04-09
406,04/10/2021,Richmond,New York,36085.0,149,3.0,2021-04-10
407,04/11/2021,Richmond,New York,36085.0,129,2.0,2021-04-11
408,04/12/2021,Richmond,New York,36085.0,204,1.0,2021-04-12


In [46]:
print (data_date_dt)

2021-05-26 00:00:00


In [47]:

covid_death = covid_death[['fips', 'deaths']]
covid_death.head(2)

,fips,deaths
1352218,1001.0,110.0
1352219,1003.0,310.0


In [48]:
Kings_death.head(2)

,fips,deaths
0,36047.0,8560


In [49]:
# We merge the NY data with the rest of the US data
covid_death = pd.concat([covid_death, Kings_death, Bronx_death, Manhattan_death, Queens_death, Richmond_death], sort = False)

covid_death = covid_death.rename(columns={"deaths": "total_deaths"})


# Create a dictionary for the cumulative COVID deaths in each county
County_covid_death = dict(zip(covid_death.fips, covid_death.total_deaths))

covid_death.tail(250)

,fips,total_deaths
1355219,51690.0,76.0
1355220,51115.0,12.0
1355221,51117.0,67.0
1355222,51119.0,20.0
1355223,51121.0,97.0
1355224,51125.0,13.0
1355225,51127.0,18.0
1355226,51700.0,228.0
1355227,51710.0,261.0
1355228,51131.0,36.0


In [50]:
covid_death.shape

(3251, 2)

In [51]:
print (County_covid_death)

{1001.0: 110.0, 1003.0: 310.0, 1005.0: 58.0, 1007.0: 64.0, 1009.0: 139.0, 1011.0: 42.0, 1013.0: 70.0, 1015.0: 323.0, 1017.0: 123.0, 1019.0: 45.0, 1021.0: 115.0, 1023.0: 25.0, 1025.0: 61.0, 1027.0: 57.0, 1029.0: 42.0, 1031.0: 120.0, 1033.0: 137.0, 1035.0: 30.0, 1037.0: 29.0, 1039.0: 119.0, 1041.0: 57.0, 1043.0: 197.0, 1045.0: 114.0, 1047.0: 154.0, 1049.0: 188.0, 1051.0: 208.0, 1053.0: 77.0, 1055.0: 357.0, 1057.0: 62.0, 1059.0: 82.0, 1061.0: 81.0, 1063.0: 34.0, 1065.0: 78.0, 1067.0: 43.0, 1069.0: 284.0, 1071.0: 112.0, 1073.0: 1535.0, 1075.0: 37.0, 1077.0: 245.0, 1079.0: 99.0, 1081.0: 173.0, 1083.0: 154.0, 1085.0: 53.0, 1087.0: 50.0, 1089.0: 512.0, 1091.0: 65.0, 1093.0: 102.0, 1095.0: 227.0, 1097.0: 815.0, 1099.0: 41.0, 1101.0: 599.0, 1103.0: 281.0, 1105.0: 26.0, 1107.0: 62.0, 1109.0: 77.0, 1111.0: 43.0, 1113.0: 39.0, 1117.0: 253.0, 1115.0: 247.0, 1119.0: 32.0, 1121.0: 179.0, 1123.0: 153.0, 1125.0: 457.0, 1127.0: 279.0, 1129.0: 39.0, 1131.0: 30.0, 1133.0: 73.0, 2013.0: 3.0, 2016.0: 0.0, 2

In [52]:
## subset last last 15 days
covid_last15 = covid[(covid['dt']>date_N1_days_ago) & (covid['dt']<= data_date_dt)].copy()
covid_last15['dt_time_delta'] = covid_last15['dt']-data_date_dt

In [53]:
## calculate new daily cases

## sort values by county and date
covid_last15.sort_values(by=['fips','dt'],inplace=True)
## remove data with 'unknown' counties
covid_last15 = covid_last15[covid_last15['fips'].notnull()].copy()

## calculate daily difference in number of cases
covid_last15['new_cases'] = covid_last15.groupby('fips')['cases'].transform(lambda x: x.diff())
## set negative new cases to zero, this can occuer due to the disperacy in the data

covid_last15.loc[covid_last15.new_cases < 1e-6, 'new_cases'] = 0
covid_last15.sort_index(inplace = True)

In [54]:
covid_last15[covid_last15.county == 'New Haven'].head(50)

,date,county,state,fips,cases,deaths,dt,dt_time_delta,new_cases
1310320,2021-05-12,New Haven,Connecticut,9009.0,91001,2093.0,2021-05-12,-14 days,NaN
1313567,2021-05-13,New Haven,Connecticut,9009.0,91085,2094.0,2021-05-13,-13 days,84.0
1316814,2021-05-14,New Haven,Connecticut,9009.0,91189,2096.0,2021-05-14,-12 days,104.0
1320061,2021-05-15,New Haven,Connecticut,9009.0,91189,2096.0,2021-05-15,-11 days,0.0
1323307,2021-05-16,New Haven,Connecticut,9009.0,91189,2096.0,2021-05-16,-10 days,0.0
1326553,2021-05-17,New Haven,Connecticut,9009.0,91372,2104.0,2021-05-17,-9 days,183.0
1329800,2021-05-18,New Haven,Connecticut,9009.0,91382,2106.0,2021-05-18,-8 days,10.0
1333047,2021-05-19,New Haven,Connecticut,9009.0,91474,2107.0,2021-05-19,-7 days,92.0
1336294,2021-05-20,New Haven,Connecticut,9009.0,91512,2109.0,2021-05-20,-6 days,38.0
1339541,2021-05-21,New Haven,Connecticut,9009.0,91552,2111.0,2021-05-21,-5 days,40.0


In [55]:
## select just last 14 days now that we have daily new cases with 15th day as baseline
covid_last14 = covid_last15[(covid_last15['dt'] > date_N_days_ago) & (covid_last15['dt'] <= data_date_dt)].copy()


In [56]:
## group by FIPS to get case load and follow up demand values for each county
covid_last14_stats = covid_last14.groupby(['fips'])['new_cases'].sum().reset_index(name ='total_cases')

covid_last14_stats[covid_last14_stats.fips == 9009].head(5)

,fips,total_cases
310,9009.0,727.0


In [57]:
covid.tail(50)

,date,county,state,fips,cases,deaths,dt
399,04/03/2021,Richmond,New York,36085.0,201,0.0,2021-04-03
400,04/04/2021,Richmond,New York,36085.0,142,3.0,2021-04-04
401,04/05/2021,Richmond,New York,36085.0,286,5.0,2021-04-05
402,04/06/2021,Richmond,New York,36085.0,248,4.0,2021-04-06
403,04/07/2021,Richmond,New York,36085.0,227,8.0,2021-04-07
404,04/08/2021,Richmond,New York,36085.0,234,3.0,2021-04-08
405,04/09/2021,Richmond,New York,36085.0,254,1.0,2021-04-09
406,04/10/2021,Richmond,New York,36085.0,149,3.0,2021-04-10
407,04/11/2021,Richmond,New York,36085.0,129,2.0,2021-04-11
408,04/12/2021,Richmond,New York,36085.0,204,1.0,2021-04-12


In [58]:
## group by FIPS to get case load and follow up demand values for each county
#covid_death = covid.groupby(['fips'])['deaths'].sum().reset_index(name ='total_deaths')
#covid_death.head(250)

In [59]:
## group by FIPS to get case load each month and follow up demand values for each county
covid['month'] = pd.to_datetime(covid['date'])
covid['year'] = pd.to_datetime(covid['date'])


In [60]:
#covid_last14_stats_montly = covid.groupby(['fips', covid.month.dt.month, covid.year.dt.year])['deaths'].sum().reset_index() 

#covid_last14_stats_montly['cumulative_death'] = covid_last14_stats_montly.groupby(['fips'])['deaths'].cumsum(axis = 0) 
#covid_last14_stats_montly = covid_last14_stats_montly.groupby(['fips', covid_last14_stats_montly.month,  covid_last14_stats_montly.year])['deaths'].cumsum()

#covid_last14_stats_montly.head(10)

In [61]:
# adding population information from CDC svi dataset
covid_last14_stats = covid_last14_stats.reset_index()
covid_last14_stats['fips'] = covid_last14_stats['fips'].astype(int)
svi_county['FIPS'] = svi_county['FIPS'].astype(int)
covid_last14_stats = pd.merge(left = covid_last14_stats, right = svi_county[['E_TOTPOP','FIPS', 'STATE']], how = 'right', right_on = 'FIPS', left_on = 'fips' )
covid_last14_stats.fillna(0 , inplace=True)

In [62]:

covid_last14_stats.tail(5)

,index,fips,total_cases,E_TOTPOP,FIPS,STATE
3215,3219.0,72153.0,23.0,36439,72153,PUERTO RICO
3216,0.0,0.0,0.0,890,2060,ALASKA
3217,0.0,0.0,0.0,2132,2105,ALASKA
3218,0.0,0.0,0.0,689,2282,ALASKA
3219,0.0,0.0,0.0,1375,2164,ALASKA


In [63]:
now = pd.to_datetime("now")

print (now)
m_now = now.month
print (m_now)

y_now = now.year

2021-05-26 13:40:46.335757
5


In [64]:
#one_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-1) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#one_month_lag_death = dict(zip(one_month_lag_cumulative_death.fips, one_month_lag_cumulative_death.cumulative_death))

#one_month_lag_cumulative_death.head(5)


In [65]:
#two_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now-2) & (covid_last14_stats_montly['year'] == y_now)]

# Create a dictionary for the last month COVID deaths in each county
#two_month_lag_death = dict(zip(two_month_lag_cumulative_death.fips, two_month_lag_cumulative_death.cumulative_death))


#two_month_lag_cumulative_death.head(5)

In [66]:
#three_month_lag_cumulative_death =  covid_last14_stats_montly[(covid_last14_stats_montly['month'] == m_now - 3) & (covid_last14_stats_montly['year'] == y_now)]


# Create a dictionary for the 3 last month COVID deaths in each county
#three_month_lag_death = dict(zip(three_month_lag_cumulative_death.fips, three_month_lag_cumulative_death.cumulative_death))

#three_month_lag_cumulative_death.head(5)

#print (three_month_lag_death)

In [67]:

# Create a dictionary for the states of the given the county FIPS
county_of_states = dict(zip(svi_county.FIPS, svi_county.STATE))

# Create a dictionary for the name of the given the county FIPS
county_name = dict(zip(svi_county.FIPS, svi_county.COUNTY))

# Create the list for county FIPS, we consider counties as analogy to the center for community health workers
location = svi_county.FIPS.tolist() #[k for k in SVI_county] #[9001, 9003, 9005, 9007, 9009, 9011, 9013, 9015]#[k for k in SVI_county]




In [68]:
for j in County_covid_death:
       
    print (j, County_covid_death[j])

1001.0 110.0
1003.0 310.0
1005.0 58.0
1007.0 64.0
1009.0 139.0
1011.0 42.0
1013.0 70.0
1015.0 323.0
1017.0 123.0
1019.0 45.0
1021.0 115.0
1023.0 25.0
1025.0 61.0
1027.0 57.0
1029.0 42.0
1031.0 120.0
1033.0 137.0
1035.0 30.0
1037.0 29.0
1039.0 119.0
1041.0 57.0
1043.0 197.0
1045.0 114.0
1047.0 154.0
1049.0 188.0
1051.0 208.0
1053.0 77.0
1055.0 357.0
1057.0 62.0
1059.0 82.0
1061.0 81.0
1063.0 34.0
1065.0 78.0
1067.0 43.0
1069.0 284.0
1071.0 112.0
1073.0 1535.0
1075.0 37.0
1077.0 245.0
1079.0 99.0
1081.0 173.0
1083.0 154.0
1085.0 53.0
1087.0 50.0
1089.0 512.0
1091.0 65.0
1093.0 102.0
1095.0 227.0
1097.0 815.0
1099.0 41.0
1101.0 599.0
1103.0 281.0
1105.0 26.0
1107.0 62.0
1109.0 77.0
1111.0 43.0
1113.0 39.0
1117.0 253.0
1115.0 247.0
1119.0 32.0
1121.0 179.0
1123.0 153.0
1125.0 457.0
1127.0 279.0
1129.0 39.0
1131.0 30.0
1133.0 73.0
2013.0 3.0
2016.0 0.0
2020.0 174.0
2050.0 22.0
2997.0 0.0
2068.0 0.0
2070.0 2.0
2090.0 43.0
2100.0 0.0
2110.0 3.0
2122.0 19.0
2130.0 1.0
2150.0 6.0
2158.0 4.0
217

20017.0 3.0
20019.0 6.0
20021.0 53.0
20023.0 14.0
20025.0 7.0
20027.0 29.0
20029.0 23.0
20031.0 23.0
20033.0 9.0
20035.0 108.0
20037.0 83.0
20039.0 12.0
20041.0 45.0
20043.0 23.0
20045.0 87.0
20047.0 12.0
20049.0 5.0
20051.0 72.0
20053.0 26.0
20055.0 72.0
20057.0 75.0
20059.0 42.0
20061.0 40.0
20063.0 19.0
20065.0 9.0
20067.0 18.0
20069.0 14.0
20071.0 1.0
20073.0 11.0
20075.0 1.0
20077.0 18.0
20079.0 69.0
20081.0 9.0
20083.0 8.0
20085.0 16.0
20087.0 35.0
20089.0 6.0
20091.0 766.0
20093.0 10.0
20095.0 15.0
20097.0 5.0
20099.0 50.0
20101.0 7.0
20103.0 91.0
20105.0 5.0
20107.0 12.0
20109.0 6.0
20111.0 85.0
20115.0 24.0
20117.0 26.0
20113.0 89.0
20119.0 12.0
20121.0 42.0
20123.0 18.0
20125.0 82.0
20127.0 22.0
20129.0 9.0
20131.0 55.0
20133.0 35.0
20135.0 16.0
20137.0 27.0
20139.0 26.0
20141.0 12.0
20143.0 13.0
20145.0 13.0
20147.0 24.0
20149.0 30.0
20151.0 25.0
20153.0 7.0
20155.0 139.0
20157.0 15.0
20159.0 12.0
20161.0 43.0
20163.0 11.0
20165.0 14.0
20167.0 28.0
20169.0 140.0
20171.0 24.0

42035.0 66.0
42037.0 133.0
42039.0 149.0
42041.0 522.0
42043.0 548.0
42045.0 1437.0
42047.0 39.0
42049.0 414.0
42051.0 317.0
42053.0 21.0
42055.0 370.0
42057.0 15.0
42059.0 40.0
42061.0 132.0
42063.0 175.0
42065.0 98.0
42067.0 86.0
42069.0 471.0
42071.0 1148.0
42073.0 210.0
42075.0 287.0
42077.0 853.0
42079.0 808.0
42081.0 284.0
42083.0 72.0
42085.0 257.0
42087.0 178.0
42089.0 315.0
42091.0 1711.0
42093.0 66.0
42095.0 711.0
42097.0 357.0
42099.0 100.0
42101.0 3645.0
42103.0 54.0
42105.0 23.0
42107.0 406.0
42109.0 84.0
42111.0 210.0
42113.0 20.0
42115.0 62.0
42117.0 108.0
42119.0 87.0
42121.0 96.0
42123.0 103.0
42125.0 300.0
42127.0 80.0
42129.0 763.0
42131.0 53.0
42133.0 807.0
72001.0 nan
72003.0 nan
72005.0 nan
72007.0 nan
72009.0 nan
72011.0 nan
72013.0 nan
72015.0 nan
72017.0 nan
72019.0 nan
72021.0 nan
72023.0 nan
72025.0 nan
72027.0 nan
72029.0 nan
72031.0 nan
72033.0 nan
72035.0 nan
72037.0 nan
72039.0 nan
72041.0 nan
72043.0 nan
72045.0 nan
72047.0 nan
72049.0 nan
72051.0 nan
72

In [69]:
K_c.head()

,fips,cases,deaths
0,36047.0,1351,54


In [70]:
K = dict(zip(K_c.fips, K_c.cases))
Q = dict(zip(Q_c.fips, Q_c.cases))
B = dict(zip(B_c.fips, B_c.cases))
M = dict(zip(M_c.fips, M_c.cases))
R  = dict(zip(R_c.fips, R_c.cases))

In [71]:
print (K)

{36047.0: 1351}


In [72]:
# Create a dictionary for the county and covid cases
covid_cases_county_ny_times = dict(zip(covid_last14_stats.fips, covid_last14_stats.total_cases))
COVID_14days = {}

for j in location:
    if j in covid_cases_county_ny_times:
        COVID_14days[j] = covid_cases_county_ny_times[j] 
        print(j, covid_cases_county_ny_times[j])
        

        
    else:
        COVID_14days[j] = 0
        
        
    if j not in County_covid_death:
        print ('j',j)
        County_covid_death[j] = 0

#print (three_month_lag_death)
#print (two_month_lag_covid_death)
#print (one_month_lag_covid_death)
#for j in location:
#    if j not in three_month_lag_death:        
#        three_month_lag_death[j] = 0
        
#    if j not in two_month_lag_death:        
#        two_month_lag_death[j] = 0
        
#    if j not in one_month_lag_death:        
#        one_month_lag_death[j] = 0
        
    #if j not in County_covid_death:
    #    County_covid_death[j] = 0

35039 36.0
1001 198.0
1009 167.0
1013 47.0
1015 95.0
1017 78.0
1031 78.0
1033 30.0
1039 61.0
1043 126.0
1045 47.0
1051 166.0
1055 116.0
1067 34.0
1069 124.0
1071 45.0
1077 65.0
1079 66.0
1083 126.0
1089 496.0
1095 91.0
1097 393.0
1103 184.0
1111 36.0
1113 65.0
1115 318.0
1117 1911.0
1121 136.0
2261 31.0
4021 653.0
5009 31.0
5011 6.0
5033 37.0
5037 15.0
5045 119.0
5047 6.0
5051 142.0
5053 31.0
5063 22.0
5083 12.0
5085 76.0
5087 7.0
5115 17.0
5117 8.0
5121 32.0
5125 113.0
5131 66.0
5145 94.0
6007 271.0
6017 93.0
6023 251.0
6027 1.0
6061 277.0
6079 76.0
6089 240.0
6093 103.0
8019 12.0
8039 55.0
8047 21.0
8051 16.0
8057 5.0
8065 15.0
8067 68.0
8071 12.0
8075 40.0
8077 658.0
8083 47.0
8085 86.0
8093 33.0
8099 25.0
8101 354.0
8103 12.0
8119 77.0
8121 8.0
9005 82.0
9007 107.0
9011 191.0
9015 100.0
10001 307.0
12019 352.0
12033 284.0
12089 138.0
12101 740.0
12109 367.0
12113 190.0
13015 113.0
13039 34.0
13045 74.0
13047 77.0
13057 250.0
13073 92.0
13077 116.0
13085 26.0
13113 56.0
13115 53.0
1

51075 16.0
51077 21.0
51079 22.0
51083 10.0
51085 147.0
51089 57.0
51093 27.0
51095 43.0
51097 6.0
51099 30.0
51101 21.0
51109 17.0
51113 6.0
51127 21.0
51137 35.0
51139 27.0
51143 31.0
51155 30.0
51165 43.0
51171 58.0
51177 158.0
51181 1.0
51187 73.0
51191 63.0
51193 19.0
51197 23.0
51199 41.0
51550 173.0
51580 8.0
51735 12.0
51800 74.0
51810 251.0
51820 15.0
53003 32.0
53011 955.0
53015 221.0
53023 1.0
53027 307.0
53029 46.0
53035 336.0
53037 112.0
53041 246.0
53045 79.0
53059 11.0
53065 107.0
53067 561.0
54003 227.0
54009 22.0
54023 11.0
54025 47.0
54029 27.0
54031 27.0
54033 220.0
54035 54.0
54037 63.0
54041 64.0
54049 93.0
54051 51.0
54053 23.0
54057 53.0
54065 33.0
54067 120.0
54073 64.0
54079 114.0
54083 130.0
54085 26.0
54091 21.0
54095 23.0
54097 37.0
54103 36.0
54107 101.0
55003 24.0
55005 55.0
55009 219.0
55011 10.0
55015 38.0
55017 60.0
55021 57.0
55023 11.0
55027 82.0
55031 99.0
55035 58.0
55039 116.0
55041 2.0
55043 33.0
55045 18.0
55047 19.0
55049 17.0
55053 11.0
55055 4

53033 2647.0
53075 34.0
54021 19.0
54045 71.0
54087 15.0
54101 23.0
55079 948.0
1023 7.0
1035 7.0
1091 9.0
1099 11.0
j 2282
5039 3.0
5041 2.0
5069 38.0
5079 2.0
5093 30.0
5113 12.0
5133 6.0
6001 788.0
6013 713.0
6037 2851.0
6053 101.0
6059 785.0
6067 1243.0
6081 282.0
6083 115.0
6085 460.0
6101 60.0
6115 61.0
8001 1433.0
8021 73.0
8025 13.0
12021 654.0
12029 10.0
12107 100.0
13005 10.0
13017 7.0
13021 85.0
13069 13.0
13081 5.0
13107 14.0
13135 693.0
13165 2.0
13167 1.0
13179 62.0
13183 4.0
13235 3.0
13269 4.0
13289 7.0
13307 1.0
13313 67.0
15003 744.0
15005 0.0
16053 17.0
16073 9.0
17031 7330.0
17077 47.0
20081 0.0
20209 128.0
21043 25.0
21135 22.0
21181 7.0
21195 51.0
21231 18.0
22013 35.0
22031 33.0
22049 38.0
22069 50.0
22071 315.0
22091 29.0
24039 43.0
25013 549.0
25025 661.0
26163 3037.0
28007 4.0
28029 19.0
28031 41.0
28041 4.0
28065 3.0
28097 12.0
28099 16.0
28113 26.0
28115 24.0
29119 22.0
29215 20.0
29221 22.0
29510 335.0
31037 1.0
32009 0.0
32023 37.0
32027 5.0
34001 194.0
35

In [73]:
for j in location: 
    if j in K:
        COVID_14days[j] = K[j]
        print (j, COVID_14days[j], K[j])

    if j in Q:
        COVID_14days[j] = Q[j]
        print (j, COVID_14days[j])
        
    if j in B:
        COVID_14days[j] = B[j]
        print (j, COVID_14days[j])
        
    if j in M:
        COVID_14days[j] = M[j]
        print (j, COVID_14days[j])

    if j in R:
        COVID_14days[j] = R[j]
        print (j, COVID_14days[j])

36085 374
36061 525
36047 1351 1351
36081 1165
36005 705


In [74]:
print (covid_cases_county_ny_times)

{1001.0: 198.0, 1003.0: 408.0, 1005.0: 17.0, 1007.0: 49.0, 1009.0: 167.0, 1011.0: 5.0, 1013.0: 47.0, 1015.0: 95.0, 1017.0: 78.0, 1019.0: 16.0, 1021.0: 119.0, 1023.0: 7.0, 1025.0: 8.0, 1027.0: 11.0, 1029.0: 11.0, 1031.0: 78.0, 1033.0: 30.0, 1035.0: 7.0, 1037.0: 19.0, 1039.0: 61.0, 1041.0: 14.0, 1043.0: 126.0, 1045.0: 47.0, 1047.0: 35.0, 1049.0: 48.0, 1051.0: 166.0, 1053.0: 26.0, 1055.0: 116.0, 1057.0: 13.0, 1059.0: 9.0, 1061.0: 43.0, 1063.0: 2.0, 1065.0: 16.0, 1067.0: 34.0, 1069.0: 124.0, 1071.0: 45.0, 1073.0: 3522.0, 1075.0: 17.0, 1077.0: 65.0, 1079.0: 66.0, 1081.0: 173.0, 1083.0: 126.0, 1085.0: 24.0, 1087.0: 23.0, 1089.0: 496.0, 1091.0: 9.0, 1093.0: 36.0, 1095.0: 91.0, 1097.0: 393.0, 1099.0: 11.0, 1101.0: 506.0, 1103.0: 184.0, 1105.0: 4.0, 1107.0: 9.0, 1109.0: 20.0, 1111.0: 36.0, 1113.0: 65.0, 1115.0: 318.0, 1117.0: 1911.0, 1119.0: 7.0, 1121.0: 136.0, 1123.0: 41.0, 1125.0: 204.0, 1127.0: 34.0, 1129.0: 11.0, 1131.0: 4.0, 1133.0: 20.0, 2013.0: 2.0, 2016.0: 6.0, 2020.0: 221.0, 2050.0: 11

In [75]:
#Parameters
pro_c_s = [(i,county_of_states[i]) for i in location ]
cartesian_pro_county_state = gp.tuplelist(pro_c_s)


In [76]:
df = covid_last14_stats
df['fips'] = df['fips'].astype(int)

In [77]:
###############################################################################################
######################## END calculating different types of vulnerabilities ###################

Since we allocate CHW proportional to the county values of certain vulnaribilites within state, we need a few function to help us with the calculations. 



In [78]:
# This function return the value for the state for the given dictionary

# More specifically sum upt the values for the counties of each state

def total_state(dict_1):
    state_dict = {}
    for s in State:
        state_dict [s] = sum(float(dict_1[j]) for (j,s) in cartesian_pro_county_state.select('*', s) if j in dict_1) 
        #print (state_dict [s],s)
    return state_dict



In [79]:
# Calculte the population per state by summing up the population in each county in the state
State_pop = total_state(population_county)

In [80]:


# This function returns the ratio of the dict value for county and state of the county
def Proportional(county_level, state_level):
    
    prop = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            prop[j] = (float(county_level[j])/float(state_level[s]))    
        else:
            prop[j] = 0
                
    return prop

In [81]:
print (ACI_total)
print (ACI_total[35039])

{1001: 9804, 1003: 33019, 1005: 7332, 1007: 4860, 1009: 8954, 1011: 3100, 1013: 5004, 1015: 25387, 1017: 7391, 1019: 5162, 1021: 9558, 1023: 3433, 1025: 6557, 1027: 3375, 1029: 2964, 1031: 9652, 1033: 9471, 1035: 2710, 1037: 2234, 1039: 6987, 1041: 3339, 1043: 15529, 1045: 10717, 1047: 13248, 1049: 17983, 1051: 12740, 1053: 6301, 1055: 21800, 1057: 4277, 1059: 6775, 1061: 6484, 1063: 2858, 1065: 4977, 1067: 3485, 1069: 23013, 1071: 11372, 1073: 125514, 1075: 3015, 1077: 15659, 1079: 6971, 1081: 23354, 1083: 15392, 1085: 3457, 1087: 5200, 1089: 48472, 1091: 4803, 1093: 6723, 1095: 21817, 1097: 93203, 1099: 5664, 1101: 52425, 1103: 21849, 1105: 3094, 1107: 4752, 1109: 6901, 1111: 4507, 1113: 13627, 1115: 13737, 1117: 19953, 1119: 4144, 1121: 19136, 1123: 9809, 1125: 34475, 1127: 13645, 1129: 4400, 1131: 4082, 1133: 5781, 2013: 451, 2016: 439, 2020: 51594, 2050: 9063, 2060: 120, 2068: 145, 2070: 2082, 2090: 13016, 2100: 731, 2105: 512, 2110: 4856, 2122: 12129, 2130: 2919, 2150: 2451, 2158

In [82]:
#ACI
ACI_State = total_state(ACI_total)

In [83]:
# Create dicts for the variables of SVI
E_POV = dict(zip(svi_county.FIPS, svi_county.EP_POV))
E_UNEMP = dict(zip(svi_county.FIPS, svi_county.EP_UNEMP))
E_PCI = dict(zip(svi_county.FIPS, svi_county.EP_PCI))
E_NOHSDP = dict(zip(svi_county.FIPS, svi_county.EP_NOHSDP))
E_AGE65 = dict(zip(svi_county.FIPS, svi_county.EP_AGE65))
E_AGE17 = dict(zip(svi_county.FIPS, svi_county.EP_AGE17))
E_DISABL = dict(zip(svi_county.FIPS, svi_county.EP_DISABL))
E_SNGPNT = dict(zip(svi_county.FIPS, svi_county.EP_SNGPNT))
E_MINRTY = dict(zip(svi_county.FIPS, svi_county.EP_MINRTY))
E_LIMENG = dict(zip(svi_county.FIPS, svi_county.EP_LIMENG))
E_MUNIT = dict(zip(svi_county.FIPS, svi_county.EP_MUNIT))
E_MOBILE = dict(zip(svi_county.FIPS, svi_county.EP_MOBILE))
E_CROWD = dict(zip(svi_county.FIPS, svi_county.EP_CROWD))
E_NOVEH = dict(zip(svi_county.FIPS, svi_county.EP_NOVEH))
E_GROUPQ = dict(zip(svi_county.FIPS, svi_county.EP_GROUPQ))


# Calculate the state value for the SVI variables
E_POV_State = total_state(E_POV)
E_UNEMP_State = total_state(E_UNEMP) 
E_PCI_State = total_state(E_PCI)
E_NOHSDP_State = total_state(E_NOHSDP) 
E_AGE65_State = total_state(E_AGE65)
E_AGE17_State = total_state(E_AGE17)
E_DISABL_State = total_state(E_DISABL)
E_SNGPNT_State = total_state(E_SNGPNT)
E_MINRTY_State = total_state(E_MINRTY)
E_LIMENG_State = total_state(E_LIMENG)
E_MUNIT_State = total_state(E_MUNIT)
E_MOBILE_State = total_state(E_MOBILE)
E_CROWD_State = total_state(E_CROWD)
E_NOVEH_State = total_state(E_NOVEH)
E_GROUPQ_State = total_state(E_GROUPQ)



# Calculate the proportinal values for the SVI variables
E_POV_Prop = Proportional(E_POV, E_POV_State )
E_UNEMP_Prop = Proportional(E_UNEMP, E_UNEMP_State ) 
E_PCI_Prop = Proportional(E_PCI, E_PCI_State )
E_NOHSDP_Prop = Proportional(E_NOHSDP, E_NOHSDP_State ) 
E_AGE65_Prop = Proportional(E_AGE65, E_AGE65_State )
E_AGE17_Prop = Proportional(E_AGE17, E_AGE17_State )
E_DISABL_Prop = Proportional(E_DISABL, E_DISABL_State )
E_SNGPNT_Prop = Proportional(E_SNGPNT, E_SNGPNT_State )
E_MINRTY_Prop = Proportional(E_MINRTY, E_MINRTY_State )
E_LIMENG_Prop = Proportional(E_LIMENG, E_LIMENG_State )
E_MUNIT_Prop = Proportional(E_MUNIT, E_MUNIT_State )
E_MOBILE_Prop = Proportional(E_MOBILE, E_MOBILE_State )
E_CROWD_Prop = Proportional(E_CROWD, E_CROWD_State )
E_NOVEH_Prop = Proportional(E_NOVEH, E_NOVEH_State )
E_GROUPQ_Prop = Proportional(E_GROUPQ, E_GROUPQ_State )

ACI_Prop = Proportional(ACI_total, ACI_State)


# SVI calculation 

We calculate the ratio of county value to state value by population for each SVI variables (we use EP-estimate percentage- values in the CDC data set), then we take the average of all 15 SVI variables. 

Let SVI variable set be K, where  

K = { Below Poverty, Unemployed, Income, No High School Diploma, Aged 65 or Older, Aged 17 or Younger, Civilian with a Disability, Single-Parent Households, Minority, Speaks English “Less than Well”, Multi-Unit Structures, Mobile Homes, Crowding, No Vehicle, Group Quarters }

We will use these variables in a county base and state base. While County base values are exactly same as the estimated values for these variables in the CDC website, to calculate the state base, we simply sum the county values for all of the counties in each state for each variable. Then we use the following formula to calculate the SVI value for each county.

Let $S$ is the set of states and $j$ is a county in the state $s$, where $s \in S$, $c^k_j$ SVI variable $k \in K$ value for county j, and $c_s$ SVI variable value for state s.

$SVI_j = \frac{1}{15}\sum_{k \in K} \frac{c^k_j}{c^k_s}$


In [84]:

from collections import Counter
# Sum all SVI variable values for each county
SVI_county_sum = dict(Counter(E_POV_Prop) + Counter(E_UNEMP_Prop) + Counter(E_PCI_Prop) + Counter(E_NOHSDP_Prop) + Counter(E_AGE65_Prop) + Counter(E_AGE17_Prop) + Counter(E_DISABL_Prop) + Counter(E_SNGPNT_Prop) + Counter(E_MINRTY_Prop) + Counter(E_LIMENG_Prop) + Counter(E_MUNIT_Prop) + Counter(E_MOBILE_Prop) + Counter(E_CROWD_Prop) + Counter(E_NOVEH_Prop) + Counter(E_GROUPQ_Prop))

# Divide the sum of all SVI variable values
SVI_county = {j: SVI_county_sum[j]/15 for j in SVI_county_sum }


# Proportional Allocation

We consider allocating 1 million CHW over the states proportional to Medicaid enrollment in each state. Further, we allocate CHW to counties in each state proportional to different county vulnerability criterias as follow.

- MEDICAID
- SVI
- YPLL
- UNEMPLOYMENT
- LAST 14 DAYS COVID CASES
- LAST 14 DAYS COVID CASES / POP
- COVID DEATHS / POP

To calculate the total number of allocated CHW to per county according to these vulnerability criterias, we define the following function called "Proportional_allocation", in which we multiply the CHW allocated to each state with the ratio of the chosen vulnerability criteria of the county to the chosen vulnerability criteria of the state, the function return a dictionary with the counties as keys and the number of CHW allocated to each county for the chosen vulnerability criteria as values. 

In [85]:
def Proportional_allocation(county_level, state_level, state_budget):
    prop_allocate = {}
       
    for (j,s) in cartesian_pro_county_state:
        if state_level[s] >= 1e-6 and j in county_level:
            #print (j,s, county_level[j],state_level[s], state_budget[s])
            prop_allocate[j,s] = (float(county_level[j])/float(state_level[s]))*float(state_budget[s])
        
        else:
            prop_allocate[j,s] = 0
            
    
    return prop_allocate

In [86]:
Medicaid_dem = Proportional_allocation(ACI_total, ACI_State, Medicaid_state)
Medicaid_demand = {m[0]: Medicaid_dem[m] for m in Medicaid_dem}
for s in State:
#    print (s)
#    print ( ACI_State[s])
    print (s,Medicaid_state[s])
#for m in Medicaid_demand:
#    print (m, Medicaid_demand[m])

NEW MEXICO 772102
ALABAMA 954438
ALASKA 231145
ARIZONA 1839932
ARKANSAS 830467
CALIFORNIA 11847711
COLORADO 1337805
CONNECTICUT 874974
DELAWARE 239009
FLORIDA 3892552
GEORGIA 1928703
IDAHO 340742
ILLINOIS 2987496
INDIANA 1602976
IOWA 699741
KANSAS 401103
KENTUCKY 1416013
LOUISIANA 1585024
MAINE 232455
MARYLAND 1372695
MASSACHUSETTS 1616404
MICHIGAN 2439425
MINNESOTA 1085778
MISSISSIPPI 632427
MISSOURI 923641
MONTANA 257006
NEBRASKA 254159
NEVADA 685073
NEW HAMPSHIRE 193436
NEW JERSEY 1759653
NEW YORK 6263164
NORTH CAROLINA 1851558
NORTH DAKOTA 96757
OHIO 2788134
OKLAHOMA 797220
OREGON 1053931
PENNSYLVANIA 3069309
RHODE ISLAND 305208
SOUTH CAROLINA 1048276
SOUTH DAKOTA 114059
TENNESSEE 1489536
TEXAS 4457644
UTAH 338812
VERMONT 161049
VIRGINIA 1497770
WASHINGTON 1790395
WEST VIRGINIA 521290
WISCONSIN 1112844
WYOMING 59302
HAWAII 351337
DISTRICT OF COLUMBIA 250860
PUERTO RICO 1622194


In [87]:
#print ('Med', Medicaid_demand[35039])
#print ('Med', Medicaid_demand[4017], 'Pop', population_county[4017], 'Med_capita', 100000*(Medicaid_demand[4017]/population_county[4017]))

County_covid_death[72049]

nan

In [88]:
#Further we create additional vulnerability values by considering SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita and the number of Medicaid enrolles in each county together

Covid_capita = {j: 100000*(COVID_14days[j]/population_county[j]) for j in location}

Covid_death_capita = {j: 100000*(County_covid_death[j]/population_county[j]) for j in location}

Medicaid_capita = {j: 100000*(Medicaid_demand[j]/population_county[j]) for j in location}

Unemployment_capita = dict(zip(df_unemp.FIPS, df_unemp.Unemployed_Rate))

#Three_month_lag_covid_death_capita = {j: 100000*(three_month_lag_death[j]/population_county[j]) for j in location}

#Two_month_lag_covid_death_capita = {j: 100000*(two_month_lag_death[j]/population_county[j]) for j in location}

#One_month_lag_covid_death_capita = {j: 100000*(one_month_lag_death[j]/population_county[j]) for j in location}

In [89]:
for j in location:
    print (j, County_covid_death[j], population_county[j], Covid_death_capita[j])

35039 75.0 38921 192.6980293414866
1001 110.0 55869 196.88915140775742
1009 139.0 57826 240.37630131774637
1013 70.0 19448 359.9341834635952
1015 323.0 113605 284.3184718982439
1017 123.0 33254 369.88031515005713
1031 120.0 52342 229.2613962019029
1033 137.0 55241 248.00419977914953
1039 119.0 37049 321.19625361008394
1043 197.0 83768 235.17333588004968
1045 114.0 49172 231.83925811437402
1051 208.0 81209 256.1292467583642
1055 357.0 102268 349.08280204951694
1067 43.0 17205 249.92734670154027
1069 284.0 105882 268.2231162992765
1071 112.0 51626 216.94495021888196
1077 245.0 92729 264.21076470144186
1079 99.0 32924 300.6925039484874
1083 154.0 98915 155.68922812515797
1089 512.0 372909 137.29891206701902
1095 227.0 96774 234.56713580093827
1097 815.0 413210 197.2362721134532
1103 281.0 119679 234.79474260313006
1111 43.0 22722 189.24390458586393
1113 39.0 57961 67.28662376425527
1115 247.0 89512 275.94065600143
1117 253.0 217702 116.21390708399557
1121 179.0 79978 223.81154817574836
22

30017 18.0 11402 157.8670408700228
30021 30.0 8613 348.31069313827936
30025 2.0 2846 70.27406886858749
30027 31.0 11050 280.54298642533934
30029 95.0 103806 91.51686800377627
30031 63.0 114434 55.0535679955258
30033 2.0 1258 158.9825119236884
30043 8.0 12221 65.46109156370183
30045 1.0 2007 49.82561036372695
30047 36.0 30458 118.19554796769323
30049 88.0 69432 126.74271229404309
30059 7.0 1862 375.9398496240601
30063 94.0 119600 78.59531772575251
30067 20.0 16606 120.43839576056847
30069 1.0 487 205.3388090349076
30071 16.0 3954 404.6535154274153
30073 4.0 5911 67.67044493317545
30075 6.0 1682 356.718192627824
30081 45.0 43806 102.72565401999726
30083 17.0 10803 157.36369526983245
30091 5.0 3309 151.1030522816561
30095 14.0 9642 145.1980916822236
30099 6.0 6147 97.60858955588093
30103 1.0 696 143.67816091954023
30105 18.0 7396 243.37479718766903
30107 8.0 2126 376.29350893697085
30111 266.0 161300 164.91010539367636
31001 46.0 31363 146.669642572458
31003 7.0 6298 111.14639568116863
31

8014 75.0 70465 106.43581920102179
8015 25.0 20356 122.81391235999213
8017 6.0 1831 327.6897870016384
8027 1.0 5068 19.73164956590371
8029 63.0 31162 202.1693087735062
8035 258.0 351154 73.47203790929336
8041 830.0 720403 115.21329033887977
8043 58.0 47839 121.23999247475909
8049 6.0 15734 38.13397737384009
8059 781.0 582881 133.98961366042124
8061 3.0 1406 213.37126600284498
8063 11.0 7097 154.99506833873468
8069 241.0 356899 67.52610682574061
8079 1.0 769 130.03901170351105
8081 22.0 13283 165.62523526311827
8091 3.0 4952 60.58158319870759
8095 13.0 4265 304.8065650644783
8097 4.0 17767 22.51364889964541
8107 20.0 25638 78.00920508620017
8113 0.0 8179 0.0
8115 2.0 2248 88.9679715302491
8123 344.0 324492 106.01185853580367
9013 187.0 150721 124.07030208132909
10003 823.0 558753 147.2922740459559
10005 493.0 234225 210.48137474650443
12001 281.0 269043 104.44427099013912
12005 396.0 174705 226.66781145359317
12009 894.0 601942 148.51929255642568
12015 437.0 188910 231.3270869726325
120

28073 88.0 63343 138.92616390129928
28075 240.0 74125 323.77740303541316
28077 24.0 12586 190.68806610519624
28085 111.0 34153 325.0080520012883
28131 33.0 18336 179.9738219895288
28137 84.0 28321 296.5996963384061
28139 68.0 22015 308.88030888030886
28149 122.0 45381 268.83497498953307
28155 32.0 9689 330.2714418412633
29005 6.0 5143 116.663426015944
29009 47.0 35789 131.32526754030567
29017 13.0 12133 107.14580070881067
29019 116.0 180463 64.27910430392933
29027 50.0 44743 111.7493239165903
29035 11.0 5982 183.88498829822802
29053 27.0 17709 152.46484838217856
29065 17.0 15573 109.16329544724844
29067 29.0 13185 219.94690936670457
29077 471.0 293086 160.70368424285022
29093 7.0 10125 69.1358024691358
29095 458.0 703011 65.14834049538342
29101 56.0 54062 103.58477303836337
29111 19.0 9776 194.35351882160393
29117 42.0 15227 275.8258356866093
29123 16.0 12088 132.36267372600926
29135 32.0 16132 198.3635011157947
29147 25.0 22092 113.16313597682418
29163 26.0 18302 142.06097694241066
29

51145 17.0 29652 57.33171455551059
51147 34.0 22802 149.1097272169108
51149 30.0 38353 78.22073892524705
51157 2.0 7370 27.137042062415198
51159 15.0 9023 166.24182644353317
51161 145.0 94186 153.95069330898434
51163 46.0 22573 203.78328091082264
51169 59.0 21566 273.5787814151906
51173 94.0 30104 312.25086367260167
51179 79.0 152882 51.673839954997966
51185 70.0 40595 172.4350289444513
51570 61.0 17370 351.1801957397812
51610 9.0 14617 61.57214202640761
51670 75.0 22529 332.9042567357628
51790 71.0 24932 284.77458687630354
53005 221.0 204390 108.12662067615834
53007 67.0 77200 86.78756476683938
53009 10.0 77331 12.931424655054247
53031 3.0 32221 9.3106979919928
53039 10.0 22425 44.593088071348944
53043 8.0 10939 73.13282749794314
53053 683.0 904980 75.4712811332847
53055 0.0 17582 0.0
53061 582.0 822083 70.7957712299123
53063 643.0 522798 122.99205429248008
53071 65.0 60760 106.97827518104015
53073 98.0 229247 42.74865101833394
54001 28.0 16441 170.30594246092087
54007 3.0 13957 21.49

47049 47.0 18523 253.73859525994706
47057 50.0 23320 214.4082332761578
47075 62.0 17304 358.29865926953306
47083 33.0 8201 402.38995244482385
47087 36.0 11786 305.44714067537757
47121 25.0 12422 201.25583641925618
47171 49.0 17883 274.00324330369625
47173 34.0 19972 170.238333667134
47181 34.0 16673 203.92250944641037
48001 126.0 57735 218.23850350740452
48015 37.0 30032 123.20191795418222
48019 27.0 23112 116.82242990654204
48021 95.0 88723 107.07482839849871
48027 434.0 362924 119.5842655762639
48033 2.0 654 305.81039755351685
48039 513.0 374264 137.06902079815316
48051 48.0 18443 260.2613457680421
48063 43.0 13094 328.3946845883611
48073 142.0 52646 269.7260950499563
48075 17.0 7306 232.68546400218997
48081 13.0 3387 383.82049010924123
48085 832.0 1034730 80.40744928628725
48089 32.0 21493 148.88568371097568
48099 86.0 75951 113.23089886900765
48123 59.0 20160 292.6587301587302
48139 314.0 184826 169.8895177085475
48171 50.0 26988 185.26752630798873
48193 27.0 8461 319.1112161683016

32023 107.0 46523 229.99376652408486
32027 20.0 6725 297.3977695167286
34001 680.0 263670 257.8981302385558
35041 59.0 18500 318.9189189189189
36061 3750.0 1628706 230.2441324585284
37005 5.0 11137 44.89539373260303
37007 55.0 24446 224.98568272928085
37065 111.0 51472 215.65122785203607
37093 56.0 55234 101.38682695441214
37107 117.0 55949 209.11901910668644
37163 112.0 63531 176.29188899907132
37181 87.0 44535 195.35197036039068
37187 31.0 11580 267.7029360967185
39075 101.0 43960 229.75432211101003
40091 59.0 19596 301.0818534394774
41045 63.0 30571 206.0776552942331
45025 105.0 45650 230.01095290251916
45027 114.0 33745 337.82782634464365
45049 51.0 19222 265.3209863697846
45065 28.0 9463 295.88925287963644
47029 100.0 36004 277.7469170092212
47129 39.0 21403 182.21744615240854
48055 98.0 43664 224.4411872480762
48105 15.0 3464 433.02540415704385
48115 69.0 12728 542.1118793211816
48137 6.0 1932 310.5590062111801
48165 44.0 21492 204.72734040573238
48273 93.0 30680 303.129074315515

48489 88.0 21358 412.0235977151419
53001 23.0 19983 115.09783315818447
54047 25.0 17624 141.8520199727644
1085 53.0 9726 544.931112482007
2158 4.0 8314 48.11161895597787
6025 735.0 181215 405.5955632811853
12049 45.0 26937 167.0564650851988
12051 85.0 42022 202.27499881014708
13037 15.0 6189 242.36548715462916
13265 3.0 1537 195.1854261548471
22065 40.0 10951 365.26344626061547
28055 6.0 1327 452.14770158251696
28123 74.0 28124 263.12046650547575
28135 41.0 13809 296.9078137446593
28157 31.0 8630 359.21205098493624
35031 477.0 71367 668.3761402328807
36005 5561.0 1418207 392.1148323199646
37079 44.0 21069 208.8376287436518
45069 56.0 26118 214.4115169614825
46121 31.0 10177 304.6084307752776
46137 9.0 2756 326.5602322206096
48047 36.0 7093 507.5426476808121
48229 12.0 4886 245.59967253376993
48323 356.0 58722 606.2463812540445
48427 298.0 64633 461.06478114894867
28021 30.0 8988 333.77837116154876
45005 15.0 8688 172.65193370165744
46102 49.0 14177 345.6302461733794
48131 41.0 11157 36

In [90]:
County_covid_death[9009], population_county[9009], Covid_death_capita[9009]

(2113.0, 854757, 247.20476111924208)

In [91]:
total_state(Covid_death_capita) 

{'NEW MEXICO': 7430.036932883404,
 'ALABAMA': 18396.768547506435,
 'ALASKA': 802.8474838859413,
 'ARIZONA': 4648.629539239023,
 'ARKANSAS': 16984.55940153741,
 'CALIFORNIA': 6091.045348289427,
 'COLORADO': 7650.445772470972,
 'CONNECTICUT': 1601.93086888904,
 'DELAWARE': 547.5007294292353,
 'FLORIDA': 13128.44680706662,
 'GEORGIA': 43372.03215721589,
 'IDAHO': 5075.749151201486,
 'ILLINOIS': 23426.728678975454,
 'INDIANA': 19658.091779872393,
 'IOWA': 24831.49701546028,
 'KANSAS': 27316.881346738784,
 'KENTUCKY': 21276.08929033855,
 'LOUISIANA': 17314.676739251863,
 'MAINE': 871.6499655105182,
 'MARYLAND': 3542.93315780436,
 'MASSACHUSETTS': 2983.006255937206,
 'MICHIGAN': 15639.024623745148,
 'MINNESOTA': 12754.186017473006,
 'MISSISSIPPI': 24390.581908811888,
 'MISSOURI': 17899.96404523936,
 'MONTANA': 11522.981719989666,
 'NEBRASKA': 12977.209041674301,
 'NEVADA': 2166.8461598167114,
 'NEW HAMPSHIRE': 828.5759347928145,
 'NEW JERSEY': 5727.092645031748,
 'NEW YORK': 9209.15598520338

In [92]:
# Dictionary for the total Covid per capita for each state
Total_covid_cap = total_state(Covid_capita) 


# Dictionary for the total Covid death per capita for each state
Total_covid_death_cap = total_state(Covid_death_capita) 


# Dictionary for the total Medicaid per capita for each state
Total_medicaid_cap = total_state(Medicaid_capita)


# Dictionary for the total Unemployment per capita for each state
Total_unemployment_cap = total_state(Unemployment_capita)



In [93]:
Covid_death_capita[72049]

nan

In [94]:
# Dictionaries for the different vulnerability criteria values for states

# Dictionary for total Medicaid patient numbers for each state
Medicaid_demand_state = total_state(Medicaid_demand) 

# Dictionary for total positive COVID cases for last 14 days in each state
Covid_state = total_state(COVID_14days) 

# Dictionary for total SVI values for each state
SVI_state = total_state(SVI_county) 

# Dictionary for total YPLL values for each state
YPLL_state = total_state(YPLL) 

# Dictionary for total Unemployment numbers for each state
Unemployment_state = total_state(Unemployment)  

In [95]:
print(Medicaid_demand_state)

{'NEW MEXICO': 772102.0, 'ALABAMA': 954437.9999999999, 'ALASKA': 231145.0, 'ARIZONA': 1839932.0, 'ARKANSAS': 830467.0000000003, 'CALIFORNIA': 11847711.0, 'COLORADO': 1337805.0, 'CONNECTICUT': 874974.0, 'DELAWARE': 239009.0, 'FLORIDA': 3892551.999999999, 'GEORGIA': 1928702.999999999, 'IDAHO': 340742.0000000001, 'ILLINOIS': 2987496.0000000005, 'INDIANA': 1602975.9999999995, 'IOWA': 699741.0, 'KANSAS': 401103.0000000001, 'KENTUCKY': 1416013.0000000002, 'LOUISIANA': 1585023.9999999995, 'MAINE': 232454.99999999994, 'MARYLAND': 1372695.0, 'MASSACHUSETTS': 1616403.9999999998, 'MICHIGAN': 2439425.0000000005, 'MINNESOTA': 1085778.0000000002, 'MISSISSIPPI': 632427.0000000001, 'MISSOURI': 923640.9999999998, 'MONTANA': 257005.99999999994, 'NEBRASKA': 254159.0, 'NEVADA': 685073.0, 'NEW HAMPSHIRE': 193436.0, 'NEW JERSEY': 1759653.0, 'NEW YORK': 6263164.000000002, 'NORTH CAROLINA': 1851557.999999999, 'NORTH DAKOTA': 96756.99999999999, 'OHIO': 2788133.9999999995, 'OKLAHOMA': 797219.9999999998, 'OREGON

# 1 million CHW allocation to states

We allocate 1 million CHWs to states proportional to total Medicaid enrolles in each state.

Let's $FedCHW$ represents the number of CHW will be allocated within states by the federal government, which is 1 million in our project. $TotMed$ represents the total Medicaid enrollee numbers over the US, $Med_s$ is the total Medicaid enrollee numbers in state $s \in S$, and $CHW_s$ is the total number of CHW allocated to state $ s\in S$. 

$CHW_s = FedCHW*\frac{Med_s}{TotMed}$



In [96]:
# We consider allocation of 1 million CHW all over the US
Federal_budget_CHW = 1000000

# First, we calculate the Total Medicaid enrolles all over the US
Total_federal_need = sum(Medicaid_demand_state[s] for s in State)

# Allocate the 1 million CHWs proportional to Medicaid enrolles in each state
Medicaid_budget_state = {s: (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW  for s in State}

In [97]:
for s in State:
    print (s, Medicaid_demand_state[s], Total_federal_need, Federal_budget_CHW, (Medicaid_demand_state[s]/Total_federal_need)*Federal_budget_CHW)

NEW MEXICO 772102.0 76274734.0 1000000 10122.644282181305
ALABAMA 954437.9999999999 76274734.0 1000000 12513.160648977155
ALASKA 231145.0 76274734.0 1000000 3030.4268252184265
ARIZONA 1839932.0 76274734.0 1000000 24122.43089566199
ARKANSAS 830467.0000000003 76274734.0 1000000 10887.838691118874
CALIFORNIA 11847711.0 76274734.0 1000000 155329.43058182282
COLORADO 1337805.0 76274734.0 1000000 17539.294204552716
CONNECTICUT 874974.0 76274734.0 1000000 11471.347772907344
DELAWARE 239009.0 76274734.0 1000000 3133.527807517493
FLORIDA 3892551.999999999 76274734.0 1000000 51033.30809387023
GEORGIA 1928702.999999999 76274734.0 1000000 25286.263207420678
IDAHO 340742.0000000001 76274734.0 1000000 4467.298437251845
ILLINOIS 2987496.0000000005 76274734.0 1000000 39167.5702205661
INDIANA 1602975.9999999995 76274734.0 1000000 21015.818947333195
IOWA 699741.0 76274734.0 1000000 9173.955296913917
KANSAS 401103.0000000001 76274734.0 1000000 5258.661406803465
KENTUCKY 1416013.0000000002 76274734.0 1000

In [98]:
print (Covid_state)

{'NEW MEXICO': 2191.0, 'ALABAMA': 10857.0, 'ALASKA': 758.0, 'ARIZONA': 7146.0, 'ARKANSAS': 2439.0, 'CALIFORNIA': 17073.0, 'COLORADO': 12650.0, 'CONNECTICUT': 2745.0, 'DELAWARE': 1952.0, 'FLORIDA': 35407.0, 'GEORGIA': 8765.0, 'IDAHO': 1962.0, 'ILLINOIS': 17688.0, 'INDIANA': 9288.0, 'IOWA': 2376.0, 'KANSAS': 1933.0, 'KENTUCKY': 5267.0, 'LOUISIANA': 5728.0, 'MAINE': 2389.0, 'MARYLAND': 4418.0, 'MASSACHUSETTS': 5224.0, 'MICHIGAN': 18417.0, 'MINNESOTA': 9056.0, 'MISSISSIPPI': 2768.0, 'MISSOURI': 5026.0, 'MONTANA': 1216.0, 'NEBRASKA': 1037.0, 'NEVADA': 3323.0, 'NEW HAMPSHIRE': 1531.0, 'NEW JERSEY': 5787.0, 'NEW YORK': 15969.0, 'NORTH CAROLINA': 12266.0, 'NORTH DAKOTA': 963.0, 'OHIO': 11420.0, 'OKLAHOMA': 969.0, 'OREGON': 6397.0, 'PENNSYLVANIA': 18440.0, 'RHODE ISLAND': 1513.0, 'SOUTH CAROLINA': 5977.0, 'SOUTH DAKOTA': 514.0, 'TENNESSEE': 6219.0, 'TEXAS': 25124.0, 'UTAH': 3570.0, 'VERMONT': 486.0, 'VIRGINIA': 5841.0, 'WASHINGTON': 12516.0, 'WEST VIRGINIA': 3433.0, 'WISCONSIN': 5446.0, 'WYOMIN

# Proportional allocation for different vulnerability values

Let V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}. We assume $v_j$ represent the vulnerability value for county $j \in J$, while $v_s$ represent the sum of the vulnerability values for each county in the state of county j.

$Prop_{v_j} = \frac{v_j}{v_s}*CHW_s$



In [99]:
# Calling proportional allocation function for different vulnerability criterias

# Proportional allocation according to cumulative Covid death in per capita in each county
Proportional_to_covid_death_cap = Proportional_allocation(Covid_death_capita, Total_covid_death_cap,Medicaid_budget_state)

# Propportional allocation according to Medicaid enrollee number in each county
Proportional_to_medicaid = Proportional_allocation(Medicaid_demand, Medicaid_demand_state,Medicaid_budget_state )

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_medicaid_cap = Proportional_allocation(Medicaid_capita, Total_medicaid_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases in each county
Proportional_to_covid = Proportional_allocation(COVID_14days, Covid_state, Medicaid_budget_state)

# Proportional allocation according to SVI score in each county
Proportional_to_SVI = Proportional_allocation(SVI_county, SVI_state, Medicaid_budget_state)

# Proportional allocation according to YPLL in each county
Proportional_to_YPLL = Proportional_allocation(YPLL, YPLL_state, Medicaid_budget_state)

# Proportional allocation according to Unemployment  in each county
Proportional_to_unemployment = Proportional_allocation(Unemployment, Unemployment_state, Medicaid_budget_state)

# Proportional allocation according to Medicaid enrolles per capita in each county
Proportional_to_unemployment_cap = Proportional_allocation(Unemployment_capita, Total_unemployment_cap, Medicaid_budget_state )

# Proportional allocation according to last 14 days positive COVID cases per capita in each county
Proportional_to_covid_capita = Proportional_allocation(Covid_capita, Total_covid_cap, Medicaid_budget_state)



In [100]:
cartesian_pro_county_state

<gurobi.tuplelist (3220 tuples, 2 values each):
 ( 35039 , NEW MEXICO           )
 ( 1001  , ALABAMA              )
 ( 1009  , ALABAMA              )
 ( 1013  , ALABAMA              )
 ( 1015  , ALABAMA              )
 ( 1017  , ALABAMA              )
 ( 1031  , ALABAMA              )
 ( 1033  , ALABAMA              )
 ( 1039  , ALABAMA              )
 ( 1043  , ALABAMA              )
 ( 1045  , ALABAMA              )
 ( 1051  , ALABAMA              )
 ( 1055  , ALABAMA              )
 ( 1067  , ALABAMA              )
 ( 1069  , ALABAMA              )
 ( 1071  , ALABAMA              )
 ( 1077  , ALABAMA              )
 ( 1079  , ALABAMA              )
 ( 1083  , ALABAMA              )
 ( 1089  , ALABAMA              )
 ( 1095  , ALABAMA              )
 ( 1097  , ALABAMA              )
 ( 1103  , ALABAMA              )
 ( 1111  , ALABAMA              )
 ( 1113  , ALABAMA              )
 ( 1115  , ALABAMA              )
 ( 1117  , ALABAMA              )
 ( 1121  , ALABAMA              )


In [101]:
Total_covid_death_cap


{'NEW MEXICO': 7430.036932883404,
 'ALABAMA': 18396.768547506435,
 'ALASKA': 802.8474838859413,
 'ARIZONA': 4648.629539239023,
 'ARKANSAS': 16984.55940153741,
 'CALIFORNIA': 6091.045348289427,
 'COLORADO': 7650.445772470972,
 'CONNECTICUT': 1601.93086888904,
 'DELAWARE': 547.5007294292353,
 'FLORIDA': 13128.44680706662,
 'GEORGIA': 43372.03215721589,
 'IDAHO': 5075.749151201486,
 'ILLINOIS': 23426.728678975454,
 'INDIANA': 19658.091779872393,
 'IOWA': 24831.49701546028,
 'KANSAS': 27316.881346738784,
 'KENTUCKY': 21276.08929033855,
 'LOUISIANA': 17314.676739251863,
 'MAINE': 871.6499655105182,
 'MARYLAND': 3542.93315780436,
 'MASSACHUSETTS': 2983.006255937206,
 'MICHIGAN': 15639.024623745148,
 'MINNESOTA': 12754.186017473006,
 'MISSISSIPPI': 24390.581908811888,
 'MISSOURI': 17899.96404523936,
 'MONTANA': 11522.981719989666,
 'NEBRASKA': 12977.209041674301,
 'NEVADA': 2166.8461598167114,
 'NEW HAMPSHIRE': 828.5759347928145,
 'NEW JERSEY': 5727.092645031748,
 'NEW YORK': 9209.15598520338

# Normalize values for comparison
To be able compare the different vulnerability values for each county, we normalize all vulnerability values as follows. 

Let $m_{v_s} = \min \{v_j, \text{ for county j in state s }\}$  and 
$M_{v_s} = \max \{v_j, \text{ for county j in state s }\}$.

We calculate the normalize value for each vulnerability for each county by substracting the min vulnerability in the state of the county and dividing that by the differences between max and min value of the vulnerability values in the state. Mathematical formulation for the normalization is as follows.

$N_{v_j} = \frac{v_j - m_{v_s}}{M_{v_s} - m_{v_s}}$ 

for each $v \in V$, where V = {Medicaid, SVI, YPLL, Unemployment, COVID, COVID_capita, COVID_death and COVID_death capita, SVI and Medicaid, YPLL and Medicaid, Unemployment and Medicaid, COVID and Medicaid, COVID_capita and Medicaid, COVID_death and Medicaid and COVID_death capita and Medicaid}, j is a county in each state $s \in S$.


In [102]:
#Normalize function to normalize the vulnerability values to be able to compare them

def normalize(dict_1):
    
    result = {}
    min_data = {s: min(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    max_data = {s: max(dict_1[j] for j in location if (j,s) in cartesian_pro_county_state) for s in State }
    
    for (j,s) in cartesian_pro_county_state:
        
        if (max_data[s] - min_data[s]) != 0 :
    
            result[j] = (dict_1[j] - min_data[s])/(max_data[s] - min_data[s])
        
        else:
            result[j] = 1
    
    return result 

# Percentile Rank

<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.percentileofscore.html"> The function scipy.stats.percentileofscore (a, score, kind='rank')   </a>
computes the percentile rank of a score relative to a list of scores. 
"rank": Average percentage ranking of score. In case of multiple matches, average the percentage rankings of all matching scores.

In [103]:
from scipy import stats

# Calculate percentile ranks

def percentile_ranks(data):
    x = {s: [] for s in State}

    for (j,s) in cartesian_pro_county_state:
         
        x[s].append(data[j])
    
    
    
    percentile_ranks = {i: stats.percentileofscore(x[s], data[i], 'rank') for (i,s) in cartesian_pro_county_state}

    return percentile_ranks

In [104]:
# Write timestamp 

time_stamp = time.strftime('%m-%d-%Y %H:%M:%S')
with open('Output/time_stamp.csv','w') as f:
    w = csv.writer(f)
    now = time.strftime('%m/%d/%Y %H:%M:%S')
    w.writerow(['time',now])
    

In [105]:
#print (Medicaid_demand)

In [106]:
#Write a function to order the dicts
def order_k(dict_1):
    dict_2 = {}
    for m in location:
        if m in dict_1.keys():
            dict_2[m] = dict_1[m]
        else:
            dict_2[m] = 0
    
    return dict_2
            

In [107]:


Medicaid_demand = order_k(Medicaid_demand)#{m: Medicaid_demand[m] for m in location}
COVID_14days = order_k(COVID_14days)#{m: COVID_14days[m] for m in location}
SVI_county = order_k(SVI_county)#{m: SVI_county[m] for m in location}
YPLL = order_k(YPLL)#{m: YPLL[m] for m in location}
Unemployment = order_k(Unemployment)#{m: Unemployment[m] for m in location}
Unemployment_capita = order_k(Unemployment_capita)
Covid_capita = order_k(Covid_capita)#{m: Unemployment_capita[m] for m in location}
Covid_death_capita = order_k(Covid_death_capita)

In [108]:
# Write file allocation with each strategies for each county 

Strategies = ["Medicaid_demand", "Medicaid_capita", "Covid", "SVI"
              , "YPLL","Unemployment", "Unemployment_capita", "Covid_capita",  "Covid_death_capita" ]

fieldnames = []  
fieldnames.append('County_FIPS')


SVI_values = {i:SVI_county[i] for i in location}
s_count = 1
for s in Strategies:   
    fieldnames.append('Proportional_allocation_to_' + s)
    fieldnames.append(s)
    fieldnames.append('Percentile_ranks_' + s)

    
        

writefile = 'Output/County_level_proportional_allocation_for_all_policies.csv'
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(fieldnames)
    for row in zip(location
                   , Proportional_to_medicaid.values(),         Medicaid_demand.values(),      percentile_ranks(Medicaid_demand).values()
                   , Proportional_to_medicaid_cap.values(),     Medicaid_capita.values(),      percentile_ranks(Medicaid_capita).values()
                   , Proportional_to_covid.values(),            COVID_14days.values(),         percentile_ranks(COVID_14days).values()
                   , Proportional_to_SVI.values(),              SVI_county.values(),           percentile_ranks(SVI_county).values()
                   , Proportional_to_YPLL.values(),             YPLL.values(),                 percentile_ranks(YPLL).values()
                   , Proportional_to_unemployment.values(),     Unemployment.values(),         percentile_ranks(Unemployment).values()
                   , Proportional_to_unemployment_cap.values(), Unemployment_capita.values(),  percentile_ranks(Unemployment_capita).values()
                   , Proportional_to_covid_capita.values(),     Covid_capita.values(),         percentile_ranks(Covid_capita).values()
                   , Proportional_to_covid_death_cap.values(),  Covid_death_capita.values(),   percentile_ranks(Covid_death_capita).values() ):                    
       
        writer.writerow(row)

In [109]:
writefile = 'Output/State_level_allocation.csv'

cl = ['State', 'CHW_allocation']
with open( writefile, 'w' ) as f:
    writer = csv.writer(f)                
    writer.writerow(cl)
    for row in zip( State, Medicaid_demand_state.values()):
        writer.writerow(row)